In [1]:
##THE HYBRID WITH SLIME AND IALS RESULTS ARE BETTER THAN ANY OTHER ALGORITHM TESTED BEFORE
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores, MergeTwoModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [4]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)


 21%|████████▏                             | 3880/18059 [02:44<20:02, 11.79it/s]


 40%|███████████████▎                      | 7304/18059 [06:27<07:43, 23.19it/s]


 58%|█████████████████████▌               | 10504/18059 [11:51<13:12,  9.53it/s]


 76%|████████████████████████████         | 13672/18059 [17:19<05:46, 12.65it/s]


 93%|██████████████████████████████████▎  | 16776/18059 [22:18<02:33,  8.36it/s]


100%|████████████████████████████████████▉| 18032/18059 [23:59<00:01, 24.84it/s]

In [5]:
RP3BETA_recommender = RP3betaRecommender(URM_train=URM_stacked)
RP3BETA_recommender.fit(topK=int(167.27758470128256), alpha=0.7718223336771766, beta=0.5375685111310673, implicit=True, normalize_similarity=True)


100%|████████████████████████████████████▉| 18056/18059 [24:17<00:00, 24.84it/s]

In [6]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=URM_stacked)
IALS_recommender.fit(epochs=10, num_factors=42, alpha=0.7617528864750021, reg=8.926401306541349)

IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 54.15 sec
IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 1.14 min
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 1.37 min
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 1.60 min
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 1.82 min
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 2.04 min
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 2.26 min
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 2.50 min
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 2.73 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 2.95 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 2.95 min


In [7]:
similarity_rp3 = RP3BETA_recommender.W_sparse
similarity_slime = SLIME_recommender.W_sparse
similarity_hybrid = ItemKNNSimilarityHybridRecommender(URM_train = URM_stacked,
                                                      Similarity_1 = similarity_rp3,
                                                      Similarity_2 = similarity_slime)
similarity_hybrid.fit(topK=494, alpha=0.1597863887003789 )
recommenders_IALS3 =[similarity_hybrid,IALS_recommender]
recommender = MergeTwoModelsByScores(URM_stacked, recommenders_IALS3)
recommender.fit(alpha=0.4555360263375717,beta=0.13654581079541908)

In [8]:
submission = create_submission(recommender)

100%|████████████████████████████████████▉| 18056/18059 [33:40<00:00,  8.94it/s]


In [9]:
write_submission(submission,"hybrid_RP3_KFOLD")

In [ ]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from skopt.space import Real, Integer
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

hyperparameters_range_dictionary = {
    "topK": Integer(450,550),
    "alpha": Real(0.0, 0.4)
}


recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, similarity_rp3, similarity_slime],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=ItemKNNSimilarityHybridRecommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 100
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=ItemKNNSimilarityHybridRecommender.RECOMMENDER_NAME,
                    save_model="best"
                    )